In [2]:
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling
from pyproj import CRS, Transformer, Proj, transform
import geopandas as gpd
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.pyplot as plt
import numpy as np
from rasterstats import zonal_stats
import pandas as pd

In [5]:

def calculate_per_county(data_path, shapefile_path, state, state_fips):
    with rasterio.open(data_path) as src:
        data = src.read(1)
        profile = src.profile
        x_res, y_res = src.res
    gdf = gpd.read_file(shapefile_path)
    gdf = gdf.query(f"STATEFP == '{state_fips}'")
    gdf = gdf.to_crs(src.crs)
    
    data[data < 0] = np.nan

    stats = zonal_stats(gdf, data, affine=src.transform, nodata=np.nan, stats=['count', 'mean', 'median', 'max', 'min'])
    mean_values = [stat['mean'] for stat in stats]
    median_values = [stat['median'] for stat in stats]
    max_values = [stat['max'] for stat in stats]
    counts = [stat['count'] for stat in stats]
    
    county_names = gdf['NAME'].tolist()
    county_fps = [state_fips + str(fp).zfill(3) for fp in gdf['COUNTYFP'].tolist()]

    pixel_area_m2 = x_res * y_res
    square_miles_flooded = [count * (pixel_area_m2 / 2589988.11) for count in counts]

    # Create DataFrame
    df = pd.DataFrame({
        'state_name': [state]*len(county_names),
        'county_name': county_names,
        'county_fips': county_fps,
        'mean_depth_m': mean_values,
        'median_depth_m': median_values,
        'max_depth_m': max_values,
        'square_miles_flooded': square_miles_flooded
    })

    df['mean_depth_f'] = df['mean_depth_m'] * 3.281
    df['median_depth_f'] = df['median_depth_m'] * 3.281
    df['max_depth_f'] = df['max_depth_m'] * 3.281
    print(df.head())
    return df

In [11]:
state = 'Georgia'
state_abbr = 'GA'
state_fips = '13'

data_path = f'../../data/UNSEEN/hurricane/usgs_flooding/Projections_WaterDepth/{state_abbr}_water_depth_SLR000_RP001.tif'
shapefile_path = '../../data/shapefiles/us/counties/cb_2018_us_county_500k.shp'


county_data = calculate_per_county(data_path, shapefile_path, state, state_fips)
county_data.to_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_{state_abbr}_v3.csv')


  state_name county_name county_fips  mean_depth_m  median_depth_m  \
0    Georgia     Baldwin       13009           NaN             NaN   
1    Georgia      Brooks       13027           NaN             NaN   
2    Georgia   Effingham       13103      0.329168             0.2   
3    Georgia      Fulton       13121           NaN             NaN   
4    Georgia     Hancock       13141           NaN             NaN   

   max_depth_m  square_miles_flooded  mean_depth_f  median_depth_f  \
0          NaN              0.000000           NaN             NaN   
1          NaN              0.000000           NaN             NaN   
2         5.16             15.018563          1.08          0.6562   
3          NaN              0.000000           NaN             NaN   
4          NaN              0.000000           NaN             NaN   

   max_depth_f  
0          NaN  
1          NaN  
2    16.929959  
3          NaN  
4          NaN  


In [62]:
# rps = ['000','001','020','100']
# slrs = ['000', '025']

# for rp in rps:
#     for slr in slrs:
#         county_data = calculate_per_county(
#                 data_path = f'../../data/UNSEEN/hurricane/usgs_flooding/Projections_WaterDepth_FL/RP{rp}/FL_water_depth_SLR{slr}_RP{rp}.tif',
#                 shapefile_path = '../../data/shapefiles/us/counties/cb_2018_us_county_500k.shp', 
#                 state = 'Georgia'
#             )

#         county_data.to_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR{slr}_RP{rp}_county_FL_max.csv', index=False)

In [64]:
# # concat states
# for rp in rps:
#     for slr in slrs:
#         combined = pd.concat([
#             pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR{slr}_RP{rp}_county_GA.csv'),
#             pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR{slr}_RP{rp}_county_FL.csv'),
#             pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR{slr}_RP{rp}_county_SC.csv'),
#             pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR{slr}_RP{rp}_county_NC.csv'),
#             pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR{slr}_RP{rp}_county_VA.csv')
#         ])

#         combined.to_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR{slr}_RP{rp}_county_south_east.csv', index=False)

In [12]:
combined = pd.concat([
    pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_GA_v3.csv'),
    pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_FL_v3.csv'),
    pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_VA_v3.csv'),
    pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_NC_v3.csv'),
    pd.read_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_SC_v3.csv')
        ])

combined.to_csv(f'../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_south_east_v3.csv', index=False)

In [14]:
population_df = pd.read_csv('../../../data/us_census/county_populations.csv')
population_df['combined_fips'] = population_df['state_fips'].astype(str).str.zfill(2) + population_df['county_fips'].astype(str).str.zfill(3)
#population_df = population_df[['combined_fips', 'state_fips', 'population']]


flood_df = pd.read_csv('../../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_south_east_v3.csv', dtype={'county_fips': str})
combined_df = flood_df.merge(population_df, left_on='county_fips', right_on='combined_fips', how='outer')    
combined_df

,Unnamed: 0,state_name_x,county_name_x,county_fips_x,mean_depth_m,median_depth_m,max_depth_m,square_miles_flooded,mean_depth_f,median_depth_f,max_depth_f,state_name_y,state_fips,county_name_y,county_fips_y,population,combined_fips
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama,1,Autauga County,1,58761,01001
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama,1,Baldwin County,3,233420,01003
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama,1,Barbour County,5,24877,01005
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama,1,Bibb County,7,22251,01007
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama,1,Blount County,9,59077,01009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wyoming,56,Sweetwater County,37,42079,56037
3147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wyoming,56,Teton County,39,23346,56039
3148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wyoming,56,Uinta County,41,20546,56041
3149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wyoming,56,Washakie County,43,7725,56043


In [15]:
unique_states = combined_df['state_fips'].unique()
population_relevant_states = population_df[population_df['state_fips'].isin(unique_states)]
combined_df_v2 = combined_df.merge(population_relevant_states, left_on='combined_fips', right_on='combined_fips', how='outer')    
combined_df_v2 = combined_df_v2[['state_name_y', 'county_name_y', 'combined_fips', 'state_fips_x', 'max_depth_m', 'max_depth_f', 'mean_depth_m', 'mean_depth_f', 'median_depth_m', 'median_depth_f', 'square_miles_flooded', 'population_y']]
combined_df_v2 = combined_df_v2.rename(columns={
    'state_name_y': 'state_name', 'county_name_y': 'county_name',  'state_fips_x': 'state_fips', 
     'population_y': 'population', 
    })
combined_df_v2['square_miles_flooded'] = combined_df_v2['square_miles_flooded'].replace(0, np.nan)
combined_df_v2['median_depth_f'] = combined_df_v2['median_depth_f'].replace(0, np.nan)
combined_df_v2

,state_name,county_name,combined_fips,state_fips,max_depth_m,max_depth_f,mean_depth_m,mean_depth_f,median_depth_m,median_depth_f,square_miles_flooded,population
0,Alabama,Autauga County,01001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58761
1,Alabama,Baldwin County,01003,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233420
2,Alabama,Barbour County,01005,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24877
3,Alabama,Bibb County,01007,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22251
4,Alabama,Blount County,01009,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59077
...,...,...,...,...,...,...,...,...,...,...,...,...
3146,Wyoming,Sweetwater County,56037,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42079
3147,Wyoming,Teton County,56039,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23346
3148,Wyoming,Uinta County,56041,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20546
3149,Wyoming,Washakie County,56043,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7725


In [16]:
combined_df_v2.to_csv('../../../data/UNSEEN/hurricane/usgs_flooding/preprocessed/SLR000_RP001_county_south_east_with_pop_v4.csv', index=False)


In [21]:
########################################################################### 
# Look into depths at specific lat lon locations for climate central images
###########################################################################

# florida
miami_beach_lat = 25.785490
miami_beach_lon = -80.135682
downtown_miami_lat = 25.772351
downtown_miami_lon = -80.189498
ft_lauderdale_lat = 26.118810
ft_lauderdale_lon = -80.136541
w_palm_beach_lat = 26.706311
w_palm_beach_lon = -80.054842
jacksonville_lat = 30.3451
jacksonville_lon = -81.6527

# south carolina
charleston = (32.778605, -79.932523)




In [68]:
lat, lon = 25.785428263283382, -80.1357869214073

transformer_1 = Transformer.from_crs("epsg:4326", "epsg:32617", always_xy=True)
converted_x, converted_y = transformer_1.transform(lat, lon)

print(f"Converted coordinates: X = {converted_x}, Y = {converted_y}")

transformer_2 = Transformer.from_crs("epsg:32617", "epsg:4326", always_xy=True)
reconverted_x, reconverted_y = transformer_1.transform(converted_x, converted_y)

print(f"Converted coordinates: X = {reconverted_x}, Y = {reconverted_y}")


Converted coordinates: X = 1558664.0829153392, Y = -10318894.809690941
Converted coordinates: X = inf, Y = inf


In [137]:
data_path = '../../data/UNSEEN/hurricane/usgs_flooding/Projections_WaterDepth/FL_water_depth_SLR000_RP001.tif'


In [140]:
from pyproj import Transformer

# Define the input coordinates (latitude and longitude)
lat, lon = 26.010284, -80.117423

# Initialize the transformer to convert from EPSG:4326 to EPSG:32617
transformer_1 = Transformer.from_crs("epsg:4326", "epsg:32617", always_xy=True)
converted_x, converted_y = transformer_1.transform(lon, lat)  # Note the order: lon, lat

print(f"Converted coordinates: X = {converted_x}, Y = {converted_y}")

# Initialize the transformer to convert back from EPSG:32617 to EPSG:4326
transformer_2 = Transformer.from_crs("epsg:32617", "epsg:4326", always_xy=True)
reconverted_lon, reconverted_lat = transformer_2.transform(converted_x, converted_y)  # X, Y

print(f"Reconverted coordinates: Latitude = {reconverted_lat}, Longitude = {reconverted_lon}")

Converted coordinates: X = 588320.720471093, Y = 2877121.0507806097
Reconverted coordinates: Latitude = 26.010284000000002, Longitude = -80.117423


In [141]:
def find_nearest_non_missing_value(src, row, col, nodata_value, max_search_radius=100):
    for r in range(1, max_search_radius + 1):
        window = rasterio.windows.Window(col - r, row - r, 2 * r + 1, 2 * r + 1)
        data = src.read(1, window=window, masked=True)
        
        # Mask out the nodata values
        data = np.ma.masked_equal(data, nodata_value)
        
        # Find the nearest non-missing value and its index
        if data.count() > 0:
            nearest_idx = np.unravel_index(np.ma.not_equal(data, nodata_value).argmax(), data.shape)
            nearest_row, nearest_col = nearest_idx[0] + row - r, nearest_idx[1] + col - r
            return data[nearest_idx], nearest_row, nearest_col
    return None, None, None

# Converted coordinates in EPSG:32617
x, y = converted_x, converted_y

# Initialize the transformer to convert from EPSG:32617 to EPSG:4326
transformer = Transformer.from_crs("epsg:32617", "epsg:4326", always_xy=True)

# Open the raster file

with rasterio.open(data_path) as src:
    # Extract the transform and nodata value
    raster_transform = src.transform
    nodata_value = src.nodata
    
    # Convert the x, y coordinates to row, col in the raster
    row, col = rowcol(raster_transform, x, y)
    row, col = int(row), int(col)

    # Check if the row and col are within the bounds of the raster
    if 0 <= row < src.height and 0 <= col < src.width:
        nearest_value, nearest_row, nearest_col = find_nearest_non_missing_value(src, row, col, nodata_value)
        nearest_value_feet = nearest_value * 3.281
        if nearest_value is not None:
            # Convert the nearest_row and nearest_col back to x, y coordinates
            nearest_x, nearest_y = xy(raster_transform, nearest_row, nearest_col)
            
            # Convert the x, y coordinates to lat, lon
            nearest_lon, nearest_lat = transformer.transform(nearest_x, nearest_y)
            
            print(f"The nearest non-missing value at coordinates (X={x}, Y={y}) is {nearest_value}")
            print(f"The coordinates of this value in lat/lon are Latitude, Longitude = {nearest_lat}, {nearest_lon}")
        else:
            print(f"No non-missing values found within the search radius at coordinates (X={x}, Y={y}).")
    else:
        print(f"Coordinates (X={x}, Y={y}) are out of bounds for the raster.")

The nearest non-missing value at coordinates (X=588320.720471093, Y=2877121.0507806097) is 0.4399999976158142
The coordinates of this value in lat/lon are Latitude, Longitude = 26.01041980991497, -80.1175590848891


In [ ]:
## see lat lon coordinates and depths in email to cc